In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [2]:
class ConvBlock(nn.Module):
    def __init__(self,in_channels,out_channels,kernel_size,stride=1):
        super(ConvBlock,self).__init__()

        # 卷积核大小与填充大小的关系——>卷积后特征图尺寸不变
        padding=(kernel_size-1)//2

        self.conv=nn.Conv2d(
            in_channels,
            out_channels,
            kernel_size,
            stride,
            padding,
            bias=False
        )
        self.norm=nn.BatchNorm2d(out_channels)
        self.act=nn.LeakyReLU(0.1,inplace=True)
    def forward(self,x):
        x=self.conv(x)
        x=self.norm(x)
        x=self.act(x)
        return x


In [ ]:
class ResidualBlock(nn.Module):
    def __init__(self,channels):
        super(ResidualBlock,self).__init__()
        self.conv1=ConvBlock(channels,channels//2,kernel_size=1)
        self.conv2=ConvBlock(channels//2,channels,kernel_size=3)
    def forward(self,x):
        residual=x
        x=self.conv1(x)
        x=self.conv2(x)
        x=residual+x

In [4]:
class ResidualBlockGroup(nn.Module):
    def __init__(self,in_channels,out_channels,num_blocks):
        super(ResidualBlockGroup,self).__init__()

        self.downsample=ConvBlock(in_channels,out_channels,kernel_size=3,stride=2)
        self.blocks=nn.ModuleList([
            ResidualBlock(out_channels) for _ in range(num_blocks)
        ])
    def forward(self,x):
        x=self.downsample(x)
        for block in self.blocks:
            x=block(x)
        return x



In [6]:
class Darkent53(nn.Module):
    def __init__(self):
        super(Darkent53,self).__init__()

        self.conv1=ConvBlock(3,32,kernel_size=3,stride=1)

        self.group1=ResidualBlockGroup(32,64,num_blocks=1)
        self.group2=ResidualBlockGroup(64,128,num_blocks=2)
        self.group3=ResidualBlockGroup(128,256,num_blocks=8)
        self.group4=ResidualBlockGroup(256,512,num_blocks=8)
        self.group5=ResidualBlockGroup(512,1024,num_blocks=4)

    def forward(self,x):
        """
        x: (batch,3,416,416)
        return 3 个不同尺度的特征图
        """
        x=self.conv1(x) # (batch,32,416,416)
        x=self.group1(x) # (batch,64,208,208)
        x=self.group2(x) # (batch,128,104,104)

        out_52=self.group3(x) # (batch,256,52,52) ← 输出1（细节丰富）
        out_26=self.group4(out_52) # (batch,512,26,26) ← 输出2（中等语义）
        out_13=self.group5(out_26) # (batch,1024,13,13) ← 输出3（高层语义）

        return out_52,out_26,out_13



        
        

In [ ]:
class YOLOv3DetectionBlock(nn.Module):
    """
    YOLOv3 的检测块
    5 个卷积层 + 检测层
    """
    def __init__(self,in_channels,num_anchors=3,num_classes=80):
        super(YOLOv3DetectionBlock,self).__init__()

        self.conv1=ConvBlock(in_channels,in_channels//2,1)
        self.conv2=ConvBlock(in_channels//2,in_channels,3)
        self.conv3=ConvBlock(in_channels,in_channels//2,1)
        self.conv4=ConvBlock(in_channels//2,in_channels,3)
        self.conv5=ConvBlock(in_channels,in_channels//2,1)

        # 检测层前的卷积s
        self.conv6=ConvBlock(in_channels//2,in_channels,3)

        # 检测层
        self.conv7=nn.Conv2d(in_channels,num_anchors*(5+num_classes),kernel_size=1)
    def forward(self,x):
        x=self.conv1(x)
        x=self.conv2(x)
        x=self.conv3(x)
        x=self.conv4(x)
        x=self.conv5(x)

        route=x # 保存用于后续融合

        x=self.conv6(x)
        detection=self.conv7(x)

        return detection,route        

In [ ]:
class YOLOv3(nn.Module):
    """
    YOLOv3 FPN
    """
    def __init__(self,backbone,num_anchors=3,num_classes=80):
        super(YOLOv3,self).__init__()
        self.backbone=backbone
        self.num_anchors=num_anchors
        self.num_classes=num_classes

        # 大目标——语义强
        self.detection_block_1=YOLOv3DetectionBlock(1024,num_anchors,num_classes)

        # 上采样
        self.upsample_conv_1=ConvBlock(512,256,1)
        self.upsample_1=nn.Upsample(scale_factor=2,mode='nearest')

        # 中目标
        self.detection_block_2=YOLOv3DetectionBlock(768,num_anchors,num_classes)

        # 上采样
        self.upsample_conv_2=ConvBlock(384,128,1)
        self.upsample_2=nn.Upsample(scale_factor=2,mode='nearest')

        # 小目标
        self.detection_block_3=YOLOv3DetectionBlock(384,num_anchors,num_classes)

    def forward(self,x):
        """
        x: (batch,3,416,416)
        返回 3 个不同尺度的检测结果
        """
        # 骨干网络提取特征
        out_52,out_26,out_13=self.backbone(x)   
        # out_52: (batch,256,52,52)  ← 浅层，细节丰富
        # out_26: (batch,512,26,26)  ← 中层
        # out_13: (batch,1024,13,13) ← 深层，语义丰富

        # 1、13×13 检测（大目标）
        detection_13,route_13=self.detection_block_1(out_13)
        # detection_13: (batch,3*(5+80),13,13) - 检测输出
        # route_13: (batch,512,13,13) - 用于上采样

        # 13×13 → 26×26 融合
        # 上采样
        x=self.upsample_conv_1(route_13) # (batch,512,13,13)->(batch,256,13,13)
        x=self.upsample_1(x) # (batch,256,13,13)->(batch,256,26,26)

        # 融合：深层上采样 + 浅层特征
        x=torch.cat([x,out_26],dim=1) # (batch,256,26,26)+(batch,512,26,26)->(batch,768,26,26)

        # 2、中目标 26×26 检测
        detection_26,route_26=self.detection_block_2(x)
        # detection_26: (batch,3*(5+80),26,26) - 检测输出
        # route_26: (batch,384,26,26) - 用于上采样

        # 26×26 → 52×52 融合
        # 上采样
        x=self.upsample_conv_2(route_26) # (batch,384,26,26)->(batch,128,26,26)
        x=self.upsample_2(x) # (batch,128,26,26)->(batch,128,52,52)

        # 融合：中层上采样 + 浅层特征
        x=torch.cat([x,out_52],dim=1) # (batch,128,52,52)+(batch,256,52,52)->(batch,384,52,52)

        # 3、小目标 52×52 检测
        detection_52,route_52=self.detection_block_3(x)
        # detection_52: (batch,3*(5+80),52,52) - 检测输出

        return detection_13,detection_26,detection_52



        

        
        